##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Perceptrones multicapa para el reconocimiento de dígitos con las API del núcleo

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/core/mlp_core"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/core/mlp_core.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/core/mlp_core.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver el código fuente en GitHub</a> </td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/core/mlp_core.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

En este bloc de notas se utilizan las [API de bajo nivel en TensorFlow Core](https://www.tensorflow.org/guide/core) con el fin de construir un flujo de trabajo de aprendizaje automático integral para la clasificación de dígitos manuscritos con [perceptrones multicapa](https://developers.google.com/machine-learning/crash-course/introduction-to-neural-networks/anatomy) y el [conjunto de datos MNIST](http://yann.lecun.com/exdb/mnist). Visite la [Descripción general de las API del núcleo](https://www.tensorflow.org/guide/core) para obtener más información sobre TensorFlow Core y sus casos de uso específicos.

## Visión general del perceptrón multicapa (MLP)

El Perceptrón Multicapa (MLP) es un tipo de red neuronal prealimentada utilizada para abordar [problemas de clasificación multiclase](https://developers.google.com/machine-learning/crash-course/multi-class-neural-networks/video-lecture). Antes de crear un MLP, es fundamental comprender los conceptos del perceptrón, capas y funciones de activación.

Los perceptrones multicapa están formados por unidades funcionales denominadas perceptrones. La ecuación de un perceptrón es la siguiente:

$$Z = \vec{w}⋅\mathrm{X} + b$$

donde:

- $Z$: salida del perceptrón
- $\mathrm{X}$: matriz de funciones
- $\vec{w}$: vector de ponderaciones
- $b$: sesgo

Cuando estos perceptrones se apilan, forman estructuras llamadas capas densas las cuales pueden conectarse para construir una red neuronal. La ecuación de una capa densa es similar a la de un perceptrón, pero utiliza una matriz de ponderaciones y un vector de sesgos:

$$Z = \mathrm{W}⋅\mathrm{X} + \vec{b}$$

donde:

- $Z$: salida de capa densa
- $\mathrm{X}$: matriz de funciones
- $\mathrm{W}$: matriz de ponderaciones
- $\vec{b}$: vector de sesgos

En un MLP, varias capas densas están conectadas de tal forma que las salidas de una capa están totalmente conectadas a las entradas de la siguiente capa. Si se agregan funciones de activación no lineales a las salidas de las capas densas, el clasificador MLP puede aprender límites de decisión complejos y generalizar bien los datos desconocidos.

## Preparación

Importe TensorFlow, [pandas](https://pandas.pydata.org), [Matplotlib](https://matplotlib.org) y [seaborn](https://seaborn.pydata.org) para comenzar.

In [ ]:
# Use seaborn for countplot.
!pip install -q seaborn

In [ ]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import tempfile
import os
# Preset Matplotlib figure sizes.
matplotlib.rcParams['figure.figsize'] = [9, 6]

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
print(tf.__version__)
# Set random seed for reproducible results 
tf.random.set_seed(22)

## Cargar los datos

En este tutorial se utiliza el conjunto de datos [MNIST](http://yann.lecun.com/exdb/mnist), y se muestra cómo construir un modelo MLP que pueda clasificar dígitos escritos a mano. El conjunto de datos está disponible en [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/mnist).

Divida el conjunto de datos MNIST en conjuntos de entrenamiento, validación y prueba. El conjunto de validación puede utilizarse para calibrar la generalización del modelo durante el entrenamiento, de modo que el conjunto de prueba pueda servir como estimador final no sesgado del rendimiento del modelo.


In [ ]:
train_data, val_data, test_data = tfds.load("mnist", 
                                            split=['train[10000:]', 'train[0:10000]', 'test'],
                                            batch_size=128, as_supervised=True)

El conjunto de datos MNIST se compone de dígitos escritos a mano y sus correspondientes etiquetas verdaderas. Visualice un par de ejemplos que aparecen a continuación.

In [ ]:
x_viz, y_viz = tfds.load("mnist", split=['train[:1500]'], batch_size=-1, as_supervised=True)[0]
x_viz = tf.squeeze(x_viz, axis=3)

for i in range(9):
    plt.subplot(3,3,1+i)
    plt.axis('off')
    plt.imshow(x_viz[i], cmap='gray')
    plt.title(f"True Label: {y_viz[i]}")
    plt.subplots_adjust(hspace=.5)


Revise también la distribución de los dígitos en los datos de entrenamiento para verificar que cada clase está bien representada en el conjunto de datos.


In [ ]:
sns.countplot(x=y_viz.numpy());
plt.xlabel('Digits')
plt.title("MNIST Digit Distribution");

## Tratamiento previo de los datos

En primer lugar, reestructure las matrices de funciones para que sean bidimensionales aplanándolas. A continuación, reescalamos los datos para que los valores de pixeles de [0.255] se ajusten al rango de [0.1]. Este paso garantiza que los pixeles de entrada tengan distribuciones similares y contribuyan a la convergencia del entrenamiento.

In [ ]:
def preprocess(x, y):
  # Reshaping the data
  x = tf.reshape(x, shape=[-1, 784])
  # Rescaling the data
  x = x/255
  return x, y

train_data, val_data = train_data.map(preprocess), val_data.map(preprocess)

## Crear la MLP

Comience visualizando las funciones de activación [ReLU](https://developers.google.com/machine-learning/glossary#ReLU) y [Softmax](https://developers.google.com/machine-learning/glossary#softmax). Ambas funciones están disponibles en `tf.nn.relu` y `tf.nn.softmax` respectivamente. La ReLU es una función de activación no lineal que emite la entrada si es positiva y 0 en caso contrario:

$$\text{ReLU}(X) = max(0, X)$$

In [ ]:
x = tf.linspace(-2, 2, 201)
x = tf.cast(x, tf.float32)
plt.plot(x, tf.nn.relu(x));
plt.xlabel('x')
plt.ylabel('ReLU(x)')
plt.title('ReLU activation function');

La función de activación softmax es una función exponencial normalizada que convierte $m$ números reales en una distribución de probabilidad con $m$ resultados/clases. Esto es útil para predecir probabilidades de las clases a partir de la salida de una red neuronal:

$$\text{Softmax}(X) = \frac{e^{X}}{\sum_{i=1}^{m}e^{X_i}}$$

In [ ]:
x = tf.linspace(-4, 4, 201)
x = tf.cast(x, tf.float32)
plt.plot(x, tf.nn.softmax(x, axis=0));
plt.xlabel('x')
plt.ylabel('Softmax(x)')
plt.title('Softmax activation function');

### La capa densa

Se creará una clase para la capa densa. Como definición, las salidas de una capa están totalmente conectadas a las entradas de la capa siguiente en un MLP. Por lo tanto, la dimensión de entrada de una capa densa puede deducirse basándose en la dimensión de salida de su capa anterior y no necesita especificarse previamente durante su inicialización. Las ponderaciones también deben inicializarse adecuadamente para evitar que las salidas de activación sean demasiado grandes o pequeñas. Uno de los métodos de inicialización de ponderaciones más populares es el esquema de Xavier, en el que cada elemento de la matriz de ponderaciones se muestrea de la siguiente manera:

$$W_{ij} \sim \text{Uniform}(-\frac{\sqrt{6}}{\sqrt{n + m}},\frac{\sqrt{6}}{\sqrt{n + m}})$$

El vector de sesgo puede inicializarse en ceros.

In [ ]:
def xavier_init(shape):
  # Computes the xavier initialization values for a weight matrix
  in_dim, out_dim = shape
  xavier_lim = tf.sqrt(6.)/tf.sqrt(tf.cast(in_dim + out_dim, tf.float32))
  weight_vals = tf.random.uniform(shape=(in_dim, out_dim), 
                                  minval=-xavier_lim, maxval=xavier_lim, seed=22)
  return weight_vals

El método de inicialización Xavier también se puede implementar en `tf.keras.initializers.GlorotUniform`.

In [ ]:
class DenseLayer(tf.Module):

  def __init__(self, out_dim, weight_init=xavier_init, activation=tf.identity):
    # Initialize the dimensions and activation functions
    self.out_dim = out_dim
    self.weight_init = weight_init
    self.activation = activation
    self.built = False

  def __call__(self, x):
    if not self.built:
      # Infer the input dimension based on first call
      self.in_dim = x.shape[1]
      # Initialize the weights and biases
      self.w = tf.Variable(self.weight_init(shape=(self.in_dim, self.out_dim)))
      self.b = tf.Variable(tf.zeros(shape=(self.out_dim,)))
      self.built = True
    # Compute the forward pass
    z = tf.add(tf.matmul(x, self.w), self.b)
    return self.activation(z)

Después, debe crear una clase para el modelo MLP que ejecuta las capas secuencialmente. Recuerde que las variables del modelo solo están disponibles después de la primera secuencia de llamadas a capas densas debido a la inferencia de dimensiones.

In [ ]:
class MLP(tf.Module):

  def __init__(self, layers):
    self.layers = layers
   
  @tf.function
  def __call__(self, x, preds=False): 
    # Execute the model's layers sequentially
    for layer in self.layers:
      x = layer(x)
    return x

Inicialice un modelo MLP con la siguiente arquitectura:

- Siguiente paso: ReLU (784 x 700) x ReLU (700 x 500) x Softmax (500 x 10)

La función de activación softmax no debe aplicarse en el MLP. Se calcula por separado en las funciones de pérdida y predicción.

In [ ]:
hidden_layer_1_size = 700
hidden_layer_2_size = 500
output_size = 10

mlp_model = MLP([
    DenseLayer(out_dim=hidden_layer_1_size, activation=tf.nn.relu),
    DenseLayer(out_dim=hidden_layer_2_size, activation=tf.nn.relu),
    DenseLayer(out_dim=output_size)])

### Definir la función de pérdida

La función de pérdida de entropía cruzada es una gran elección para los problemas de clasificación multiclase, ya que mide la probabilidad logarítmica negativa de los datos según las predicciones de probabilidad del modelo. Mientras mayor sea la probabilidad asignada a la clase verdadera, menor será la pérdida. La ecuación para la pérdida de entropía cruzada es la siguiente:

$$L = -\frac{1}{n}\sum_{i=1}^{n}\sum_{i=j}^{n} {y_j}^{[i]}⋅\log(\hat{{y_j}}^{[i]})$$

donde:

- $\underset{n\times m}{\hat{y}}$: una matriz de distribuciones de clase previstas
- $\underset{n\times m}{y}$: una matriz codificada atractiva de clases verdaderas

La función `tf.nn.sparse_softmax_cross_entropy_with_logits` puede utilizarse para calcular la pérdida de entropía cruzada. Esta función no requiere que la última capa del modelo aplique la función de activación softmax ni que las etiquetas de las clases se codifiquen de forma atractiva.

In [ ]:
def cross_entropy_loss(y_pred, y):
  # Compute cross entropy loss with a sparse operation
  sparse_ce = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_pred)
  return tf.reduce_mean(sparse_ce)

Escriba una función básica de precisión que calcula la proporción de clasificaciones correctas durante el entrenamiento. Para generar predicciones de clase a partir de salidas softmax, devuelva el índice que corresponde a la mayor probabilidad de la clase. 

In [ ]:
def accuracy(y_pred, y):
  # Compute accuracy after extracting class predictions
  class_preds = tf.argmax(tf.nn.softmax(y_pred), axis=1)
  is_equal = tf.equal(y, class_preds)
  return tf.reduce_mean(tf.cast(is_equal, tf.float32))

### Entrenar al modelo

El uso de un optimizador podría acelerar significativamente la convergencia en comparación con el descenso mediante un gradiente estándar. El optimizador Adam se implementa a continuación. Visite la guía [Optimizers](https://www.tensorflow.org/guide/core/optimizers_core) para obtener más información sobre el diseño de optimizadores personalizados con TensorFlow Core.

In [ ]:
class Adam:

    def __init__(self, learning_rate=1e-3, beta_1=0.9, beta_2=0.999, ep=1e-7):
      # Initialize optimizer parameters and variable slots
      self.beta_1 = beta_1
      self.beta_2 = beta_2
      self.learning_rate = learning_rate
      self.ep = ep
      self.t = 1.
      self.v_dvar, self.s_dvar = [], []
      self.built = False
      
    def apply_gradients(self, grads, vars):
      # Initialize variables on the first call
      if not self.built:
        for var in vars:
          v = tf.Variable(tf.zeros(shape=var.shape))
          s = tf.Variable(tf.zeros(shape=var.shape))
          self.v_dvar.append(v)
          self.s_dvar.append(s)
        self.built = True
      # Update the model variables given their gradients
      for i, (d_var, var) in enumerate(zip(grads, vars)):
        self.v_dvar[i].assign(self.beta_1*self.v_dvar[i] + (1-self.beta_1)*d_var)
        self.s_dvar[i].assign(self.beta_2*self.s_dvar[i] + (1-self.beta_2)*tf.square(d_var))
        v_dvar_bc = self.v_dvar[i]/(1-(self.beta_1**self.t))
        s_dvar_bc = self.s_dvar[i]/(1-(self.beta_2**self.t))
        var.assign_sub(self.learning_rate*(v_dvar_bc/(tf.sqrt(s_dvar_bc) + self.ep)))
      self.t += 1.
      return 

Ahora, escriba un bucle de entrenamiento personalizado que actualice los parámetros del MLP con el descenso de gradiente en mini-lotes. El uso de minilotes para el entrenamiento proporciona eficiencia de memoria y una convergencia más rápida.

In [ ]:
def train_step(x_batch, y_batch, loss, acc, model, optimizer):
  # Update the model state given a batch of data
  with tf.GradientTape() as tape:
    y_pred = model(x_batch)
    batch_loss = loss(y_pred, y_batch)
  batch_acc = acc(y_pred, y_batch)
  grads = tape.gradient(batch_loss, model.variables)
  optimizer.apply_gradients(grads, model.variables)
  return batch_loss, batch_acc

def val_step(x_batch, y_batch, loss, acc, model):
  # Evaluate the model on given a batch of validation data
  y_pred = model(x_batch)
  batch_loss = loss(y_pred, y_batch)
  batch_acc = acc(y_pred, y_batch)
  return batch_loss, batch_acc

In [ ]:
def train_model(mlp, train_data, val_data, loss, acc, optimizer, epochs):
  # Initialize data structures
  train_losses, train_accs = [], []
  val_losses, val_accs = [], []

  # Format training loop and begin training
  for epoch in range(epochs):
    batch_losses_train, batch_accs_train = [], []
    batch_losses_val, batch_accs_val = [], []

    # Iterate over the training data
    for x_batch, y_batch in train_data:
      # Compute gradients and update the model's parameters
      batch_loss, batch_acc = train_step(x_batch, y_batch, loss, acc, mlp, optimizer)
      # Keep track of batch-level training performance
      batch_losses_train.append(batch_loss)
      batch_accs_train.append(batch_acc)

    # Iterate over the validation data
    for x_batch, y_batch in val_data:
      batch_loss, batch_acc = val_step(x_batch, y_batch, loss, acc, mlp)
      batch_losses_val.append(batch_loss)
      batch_accs_val.append(batch_acc)

    # Keep track of epoch-level model performance
    train_loss, train_acc = tf.reduce_mean(batch_losses_train), tf.reduce_mean(batch_accs_train)
    val_loss, val_acc = tf.reduce_mean(batch_losses_val), tf.reduce_mean(batch_accs_val)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f"Epoch: {epoch}")
    print(f"Training loss: {train_loss:.3f}, Training accuracy: {train_acc:.3f}")
    print(f"Validation loss: {val_loss:.3f}, Validation accuracy: {val_acc:.3f}")
  return train_losses, train_accs, val_losses, val_accs

Entrene el modelo MLP durante 10 épocas con un tamaño de lote de 128. Los aceleradores de hardware como las GPU o las TPU también pueden ayudar a acelerar el tiempo del entrenamiento. 

In [ ]:
train_losses, train_accs, val_losses, val_accs = train_model(mlp_model, train_data, val_data, 
                                                             loss=cross_entropy_loss, acc=accuracy,
                                                             optimizer=Adam(), epochs=10)

### Evaluación del desempeño

Empiece por escribir una función para graficar las pérdidas y la precisión del modelo durante el entrenamiento. 

In [ ]:
def plot_metrics(train_metric, val_metric, metric_type):
  # Visualize metrics vs training Epochs
  plt.figure()
  plt.plot(range(len(train_metric)), train_metric, label = f"Training {metric_type}")
  plt.plot(range(len(val_metric)), val_metric, label = f"Validation {metric_type}")
  plt.xlabel("Epochs")
  plt.ylabel(metric_type)
  plt.legend()
  plt.title(f"{metric_type} vs Training epochs");

In [ ]:
plot_metrics(train_losses, val_losses, "cross entropy loss")

In [ ]:
plot_metrics(train_accs, val_accs, "accuracy")

## Guardar y cargar el modelo

Comience por crear un módulo de exportación que reciba los datos sin procesar y realice las siguientes operaciones:

- Preprocesamiento de los datos
- Predicción de probabilidades
- Predicción de clases

In [ ]:
class ExportModule(tf.Module):
  def __init__(self, model, preprocess, class_pred):
    # Initialize pre and postprocessing functions
    self.model = model
    self.preprocess = preprocess
    self.class_pred = class_pred

  @tf.function(input_signature=[tf.TensorSpec(shape=[None, None, None, None], dtype=tf.uint8)]) 
  def __call__(self, x):
    # Run the ExportModule for new data points
    x = self.preprocess(x)
    y = self.model(x)
    y = self.class_pred(y)
    return y 

In [ ]:
def preprocess_test(x):
  # The export module takes in unprocessed and unlabeled data
  x = tf.reshape(x, shape=[-1, 784])
  x = x/255
  return x

def class_pred_test(y):
  # Generate class predictions from MLP output
  return tf.argmax(tf.nn.softmax(y), axis=1)

Este módulo de exportación puede guardarse ahora con la función `tf.saved_model.save`. 

In [ ]:
mlp_model_export = ExportModule(model=mlp_model,
                                preprocess=preprocess_test,
                                class_pred=class_pred_test)

In [ ]:
models = tempfile.mkdtemp()
save_path = os.path.join(models, 'mlp_model_export')
tf.saved_model.save(mlp_model_export, save_path)

Cargue el modelo guardado con `tf.saved_model.load` y examine su rendimiento en datos de prueba que no se hayan visto.

In [ ]:
mlp_loaded = tf.saved_model.load(save_path)

In [ ]:
def accuracy_score(y_pred, y):
  # Generic accuracy function
  is_equal = tf.equal(y_pred, y)
  return tf.reduce_mean(tf.cast(is_equal, tf.float32))

x_test, y_test = tfds.load("mnist", split=['test'], batch_size=-1, as_supervised=True)[0]
test_classes = mlp_loaded(x_test)
test_acc = accuracy_score(test_classes, y_test)
print(f"Test Accuracy: {test_acc:.3f}")

El modelo clasifica muy bien los dígitos escritos a mano en el conjunto de datos de entrenamiento y también generaliza bien a los datos que no se hayan visto. A continuación, examinamos la precisión por las clases del modelo para garantizar un buen rendimiento para cada dígito. 

In [ ]:
print("Accuracy breakdown by digit:")
print("---------------------------")
label_accs = {}
for label in range(10):
  label_ind = (y_test == label)
  # extract predictions for specific true label
  pred_label = test_classes[label_ind]
  labels = y_test[label_ind]
  # compute class-wise accuracy
  label_accs[accuracy_score(pred_label, labels).numpy()] = label
for key in sorted(label_accs):
  print(f"Digit {label_accs[key]}: {key:.3f}")

Parece que el modelo tiene más dificultades con algunos dígitos que con otros, lo que es bastante habitual en muchos problemas de clasificación multiclase. Como ejercicio final, grafique una matriz de confusión de las predicciones del modelo y sus correspondientes etiquetas verdaderas para obtener más información a nivel de clase. Sklearn y seaborn tienen funciones para generar y visualizar matrices de confusión. 

In [ ]:
import sklearn.metrics as sk_metrics

def show_confusion_matrix(test_labels, test_classes):
  # Compute confusion matrix and normalize
  plt.figure(figsize=(10,10))
  confusion = sk_metrics.confusion_matrix(test_labels.numpy(), 
                                          test_classes.numpy())
  confusion_normalized = confusion / confusion.sum(axis=1, keepdims=True)
  axis_labels = range(10)
  ax = sns.heatmap(
      confusion_normalized, xticklabels=axis_labels, yticklabels=axis_labels,
      cmap='Blues', annot=True, fmt='.4f', square=True)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

show_confusion_matrix(y_test, test_classes)

La información a nivel de clase puede ayudar a identificar las razones de las clasificaciones erróneas y mejorar el rendimiento del modelo en futuros ciclos de entrenamiento.

## Conclusión

Este bloc de notas introdujo algunas técnicas para hacer frente a un problema de clasificación multiclase con un [MLP](https://developers.google.com/machine-learning/crash-course/multi-class-neural-networks/softmax). Aquí encontrará algunos consejos más que pueden ayudarle:

- Las [API de TensorFlow Core](https://www.tensorflow.org/guide/core) se pueden utilizar para crear flujos de trabajo de aprendizaje automático con altos niveles de configuración.
- Los esquemas de inicialización pueden ayudar a evitar que los parámetros del modelo desaparezcan o exploten durante el entrenamiento.
- El sobreajuste es otro problema común para las redes neuronales, aunque no fue un problema para este tutorial. Visite el tutorial [Sobreajuste y subajuste](overfit_and_underfit.ipynb) para obtener más ayuda al respecto.

Para obtener más ejemplos sobre el uso de las API de TensorFlow Core, consulte la [guía](https://www.tensorflow.org/guide/core). Si desea obtener más información sobre la carga y preparación de datos, consulte los tutoriales sobre la [carga de datos de imagen](https://www.tensorflow.org/tutorials/load_data/images) o la [carga de datos CSV](https://www.tensorflow.org/tutorials/load_data/csv).